In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('todos_os_resultados.csv')

In [3]:
sum(df.duplicated())

11

In [4]:
df = df.drop_duplicates(keep="first", subset=[coluna for coluna in df.columns if coluna!="crawled_at"]).reset_index(drop=True)

In [5]:
df.shape

(3589, 13)

In [6]:
filtro_de_anuncios = [str(_id).isnumeric() for _id in df["id"]]

In [7]:
df = df[filtro_de_anuncios].reset_index(drop=True)

In [8]:
df.shape

(3589, 13)

In [9]:
set(df["rooms"])

{'-- Quarto',
 '1 Quarto',
 '10 Quartos',
 '2 Quartos',
 '3 Quartos',
 '31 Quartos',
 '4 Quartos',
 '5 Quartos',
 '6 Quartos',
 '8 Quartos',
 '9 Quartos'}

In [10]:
set(df["rooms"].str.split(" ").str[0].str.replace("--" , "0"))

{'0', '1', '10', '2', '3', '31', '4', '5', '6', '8', '9'}

In [11]:
df["Quarto"] = (df["rooms"]
                     .str.split(" ")
                     .str[0]
                     .str.replace("--","0")
                     .astype(int))

In [12]:
df["Banheiro"] = (df["bathrooms"]
                         .str.split(" ")
                         .str[0]
                         .str.replace("--","0")
                         .astype(int))

In [13]:
df["Garagem"] = (df["garages"]
                       .str.split(" ")
                       .str[0]
                       .str.replace("--","0")
                       .astype(int))

In [14]:
df["condo"] = df["condo"].fillna("MISSING")

In [15]:
df["condominio"] = [int(w.split("R$ ")[1].replace(".","")) if w!="MISSING" else np.nan for w in df["condo"]]

In [16]:
#df["price_limpo"] = [int(w.split("R$ ")[1].split(" /mês")[0].split(" /dia")[0].replace(".","")) for w in df["price"]]
df["preço"] = [int(w.split("R$ ")[1].split(" /")[0].replace(".","")) for w in df["price"]]

In [17]:
df["area_limpo"] = df["area"].astype(int)

In [18]:
df["crawled_at"] = pd.to_datetime(df["crawled_at"], format="%Y-%m-%d %H:%M")

In [19]:
df = df.drop(columns = ["area", "rooms", "bathrooms", "garages", "price", "condo"])

In [20]:
df["bairro"] = (df["address"]
                      .str.split("- ")
                      .str[1]
                      .str.split(",")
                      .str[0])

In [21]:
df.loc[df["bairro"].isin(["RJ"]), "bairro"] = (df
                                                     .loc[df["bairro"].isin(["RJ"]), "address"]
                                                     .str.split(",").str[0])

In [22]:
df.loc[df["crawler"]!=df["bairro"],"bairro"].unique()

array(['Barra da Tijuca'], dtype=object)

In [23]:
df["crawler"] = (df["crawler"]
                 .str.lower()
                 .str.replace(" ","_"))

In [24]:
df["bairro"] = (df["bairro"]
                      .str.lower()
                      .str.replace(" ","_")
                      .str.normalize("NFKD")
                      .str.encode("ascii", errors="ignore")
                      .str.decode("utf-8"))

In [25]:
df.loc[df["crawler"]!=df["bairro"], ["crawler","bairro"]]

,crawler,bairro
2869,barra,barra_da_tijuca
2870,barra,barra_da_tijuca
2871,barra,barra_da_tijuca
2872,barra,barra_da_tijuca
2873,barra,barra_da_tijuca
...,...,...
3584,barra,barra_da_tijuca
3585,barra,barra_da_tijuca
3586,barra,barra_da_tijuca
3587,barra,barra_da_tijuca


In [26]:
df = df.drop(columns="bairro")

In [27]:
df["amenities"]

0       Elevador\nAceita animais\nSalão de festas\nPor...
1                                      Condomínio fechado
2                      Condomínio fechado\nAceita animais
3       Ar-condicionado\nElevador\nBicicletário\nPortã...
4       Elevador\nAceita animais\nAcesso para deficientes
                              ...                        
3584                                                  NaN
3585    Mobiliado\nPiscina\nAr-condicionado\nCondomíni...
3586    Piscina\nChurrasqueira\nElevador\nÁrea de serv...
3587    Piscina\nChurrasqueira\nElevador\nAcademia\nCo...
3588                                                  NaN
Name: amenities, Length: 3589, dtype: object

In [28]:
df["amenities"] = df["amenities"].str.split("\n")

In [29]:
df["amenities"] = df["amenities"].fillna("")

In [30]:
amenities = []
for am in df["amenities"]:
    if am != "":
        amenities = amenities + am

In [31]:
unico_amenities = list(set(amenities))

In [32]:
df["amenities"] = df["amenities"].str.join(", ")

In [33]:
for amenity in unico_amenities:
    print(amenity)
    df[amenity]=0
    df.loc[df["amenities"].str.contains(amenity), amenity] = 1

Piscina para adulto
Cabeamento estruturado
Interfone
Churrasqueira na varanda
Segurança 24h
Sala de jantar
Aquecimento
Closet
Conexão à internet
Vista para a montanha
Vista exterior
Mais de um andar
Estacionamento
Depósito
Lavabo
Academia
Bicicletário
Estacionamento para visitantes
Salão de jogos
Piso de madeira
Perto de vias de acesso
Próximo a escola
Sala grande
Espaço teen
Condomínio fechado
Piscina
Espaço verde / Parque
Copa
Espaço gourmet
Andar inteiro
Armário na cozinha
Quadra de tênis
Próximo a hospitais
Ar-condicionado
Próximo a shopping
Cozinha
Drywall
Área de lazer
Gesso - Sanca - Teto Rebaixado
Sala de massagem
Móvel planejado
Portaria 24h
Escritório
Muro e grade
Quintal
Fogão
Câmera de segurança
Ambientes integrados
Janela grande
Quadra poliesportiva
Gás Encanado
Área de serviço
Box blindex
Varanda
Garagem
Acesso para deficientes
TV a cabo
Sauna
Sistema de alarme
Recepção
Circuito de segurança
Portão eletrônico
Playground
Vigia
Salão de festas
Armário embutido
Próximo a tra

In [39]:
df.to_csv("aptos_csv.csv", index=False)

In [41]:
pd.read_csv('todos_os_resultados.csv').shape

(3600, 13)

In [40]:
pd.read_csv("aptos_csv.csv").shape

(3589, 95)